목표: 팀별 상위 3명의 연봉 평균값 구하기
1. 연도별 employee.xlsx 파일들을 xlwings를 이용하여 읽어들인다.
2. 읽어들인 데이터를 병합하여 하나의 데이터 프레임을 생성한다.
3. result.xlsx 파일을 생성한다.
4. result.xlsx에서 team별로 sheet를 만들어준다.
5. team별 sheet의 A1셀에 각 team 연봉 상위 3명의 연봉 평균을 적어 놓는다.

    5-1. team별로 dataframe을 만들어준다.

    5-2. team별로 생성된 dataframe에서 연봉 상위 3명의 연봉을 평균낸다. (일의자리까지 반올림. 예를들어 1524.7 => 1525)

    5-3. 값을 A1셀에 적어넣는다.

6. result.xlsx을 저장한다.

In [ ]:
import os, glob
import xlwings as xw
import pandas as pd

files = glob.glob('[!~]*_employee.xlsx')
app = xw.App(visible=False) #엑셀을 최소화 된 상태로 생성
df_list = []
for file in files:
    wb = app.books.open(file) #최소화 된 엑셀에서 해당하는 파일이름의 파일 오픈
    ws = wb.sheets[0]
    df = ws.range('A1').expand().options(pd.DataFrame).value
    df_list.append(df)
concated_df = pd.concat(df_list)
concated_df.reset_index(inplace=True)
wb.close()

wb = xw.Book()

teams = concated_df['team'].unique()
for team_name in teams:
    team_df = concated_df[ (concated_df['team'] == team_name)]
    soted_df = team_df.sort_values(by='salary', ascending=False, ignore_index=True)
    result = round( soted_df.head(3)['salary'].mean() )
    ws = wb.sheets.add(team_name)
    ws.range('A1').value = result

wb.save('result.xlsx')
wb.close()
app.kill()

심화 목표: 연봉 실수령액(salary - 비용)을 내림차순으로 정렬하여 result sheet에 입력하기

(참고할점: 이름 중복 없음.)

1. 각 사원들의 구매목록중에서 support == False인 경우를 salary에서 제외 해서 연봉 실수령액을 구한다.
2. 모든 사원의 연봉 실수령액 정보를 담은 하나의 병합된 dataframe을 만든다.
3. 병합된 dataframe을 내림차순 정렬하여 result sheet에 입력한다.


In [ ]:
#--- 아래는 히든
name_list = []
team_list = []
real_salary_list = []

#app = xw.App(visible=False) #엑셀을 최소화 된 상태로 생성

for name in concated_df['name']:
    filename = './empinfo/'+name+'.xlsx'
    wb = app.books.open(filename)
    ws = wb.sheets[0]
    df = ws.range('A1').expand().options(pd.DataFrame).value
    
    salary = (concated_df[concated_df['name'] == name]['salary'].values[0]) #해당 사원의 급여
    team = (concated_df[concated_df['name'] == name]['team'].values[0]) #해당 사원의 부서

    unsupport_item_price_sum = df[ df['support'] == "FALSE" ]['price'].sum() #sum으로 처리 해도 됨.
    salary -= unsupport_item_price_sum

    name_list.append(name)
    team_list.append(team)
    real_salary_list.append(salary)
    wb.close()

result = pd.DataFrame({
    'name' : name_list,
    'team' : team_list,
    'salary' : real_salary_list
})
result = result.sort_values(by='salary', ascending=False, ignore_index=True)

wb = app.books.open('result.xlsx') #최소화 된 엑셀에서 해당하는 파일이름의 파일 오픈
ws = wb.sheets.add("result")
ws.range('A1').options(index=False).value = result
wb.save()
wb.close
app.kill()